In [ ]:
import ee
import geopandas as gpd

def shp_to_ee_geometry(shapefile_path):
    gdf = gpd.read_file(shapefile_path)
    return ee.Geometry.Polygon(gdf.geometry[0].__geo_interface__['coordinates'])

def export_composite(aoi_ee, start_date, end_date, out_name):
    s2 = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterDate(start_date, end_date)
        .filterBounds(aoi_ee)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .map(lambda img: img.divide(10000)))  # Scale reflectance

    composite = s2.median().select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'])

    ndvi = composite.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndbi = composite.normalizedDifference(['B11', 'B8']).rename('NDBI')
    ndwi = composite.normalizedDifference(['B3', 'B8']).rename('NDWI')

    stack = composite.addBands([ndvi, ndbi, ndwi])

    task = ee.batch.Export.image.toDrive(**{
        'image': stack,
        'description': out_name,
        'folder': 'ml-rf-lulc',
        'region': aoi_ee,
        'scale': 10,
        'fileFormat': 'GeoTIFF'
    })
    task.start()

# Example usage
ee.Initialize()
aoi = shp_to_ee_geometry('study-area/barishal.shp')
export_composite(aoi, '2022-01-01', '2022-03-31', 'barishal_composite_2022')
